In [1]:
# %load_ext autoreload
# %autoreload 2
import logistic

print("hello world")

hello world


# CSCI 0451: Implementing Logistic Regression


### Abstract

This post is an exploration of the Palmer Penguins dataset. The dataset contains measurements of penguins from three different species. I will use the dataset to train a machine learning model to classify the species of penguins based on certain measurements. I will primarily be utilizing the  `pandas` library, but I will also make use of the `scikit-learn` library to train a support vector machine (SVM) model on the dataset and the `matplotlib` library to visualize the data and the decision boundaries of the model.
